Dataset Rating: https://grouplens.org/datasets/movielens/32m/ <br>
[TMDB API](https://www.themoviedb.org/) Data movie: https://developer.themoviedb.org/docs/

In [12]:
import pandas as pd
import requests
import time
from IPython.display import clear_output

Import data

In [13]:
movies = pd.read_csv('./ml-32m/movies.csv')
ratings = pd.read_csv('./ml-32m/ratings.csv')
links = pd.read_csv('./ml-32m/links.csv')

Merge data in MovieLens (movies, ratings, links)

In [14]:
df = pd.merge(ratings, movies, on='movieId')
df = pd.merge(df, links, on='movieId')

In [15]:
# Drop unnecessary columns
df.drop(columns=['genres'], inplace=True)
df.drop(columns=['imdbId'], inplace=True)
df = df.reset_index(drop=True)
df

,userId,movieId,rating,timestamp,title,tmdbId
0,1,17,4.0,944249077,Sense and Sensibility (1995),4584.0
1,1,25,1.0,944250228,Leaving Las Vegas (1995),451.0
2,1,29,2.0,943230976,"City of Lost Children, The (Cité des enfants p...",902.0
3,1,30,5.0,944249077,Shanghai Triad (Yao a yao yao dao waipo qiao) ...,37557.0
4,1,32,5.0,943228858,Twelve Monkeys (a.k.a. 12 Monkeys) (1995),63.0
...,...,...,...,...,...,...
32000199,200948,79702,4.5,1294412589,Scott Pilgrim vs. the World (2010),22538.0
32000200,200948,79796,1.0,1287216292,Centurion (2010),23759.0
32000201,200948,80350,0.5,1294412671,Vampires Suck (2010),40264.0
32000202,200948,80463,3.5,1350423800,"Social Network, The (2010)",37799.0


Filter data to get only movies with year_published >= 2022

In [16]:
df['year_published'] = df['title'].str.extract(r'\((\d{4})\)', expand=False)
df['year_published'] = df['year_published'].fillna(0).astype(int)
df.drop(columns=['title'], inplace=True)

df = df[df['year_published'] > 2022]
df

,userId,movieId,rating,timestamp,tmdbId,year_published
5369,28,285593,3.0,1695439205,447365.0,2023
5370,28,286897,5.0,1692916119,569094.0,2023
5371,28,287699,5.0,1692916114,872585.0,2023
5372,28,288513,5.0,1693533442,346698.0,2023
5373,28,291485,4.0,1696391189,1059811.0,2023
...,...,...,...,...,...,...
31976553,200771,286897,4.0,1687535911,569094.0,2023
31976753,200774,287633,2.5,1693098259,747188.0,2023
31976754,200774,288513,3.0,1693098301,346698.0,2023
31983111,200822,286905,3.0,1688557693,335977.0,2023


Filter data to get only ratings with timestamp >= 2023-08-13

In [17]:
df['timestamp'] = pd.to_datetime(df['timestamp'], unit='s')
df.drop(columns=['year_published'], inplace=True)

df = df[df['timestamp'] > '2023-08-13']
df['timestamp'] = df['timestamp'].astype('int64') // 10**9
df

C:\Users\minhc\AppData\Local\Temp\ipykernel_11492\521106596.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['timestamp'] = pd.to_datetime(df['timestamp'], unit='s')
C:\Users\minhc\AppData\Local\Temp\ipykernel_11492\521106596.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(columns=['year_published'], inplace=True)
C:\Users\minhc\AppData\Local\Temp\ipykernel_11492\521106596.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in t

,userId,movieId,rating,timestamp,tmdbId
5369,28,285593,3.0,1695439205,447365.0
5370,28,286897,5.0,1692916119,569094.0
5371,28,287699,5.0,1692916114,872585.0
5372,28,288513,5.0,1693533442,346698.0
5373,28,291485,4.0,1696391189,1059811.0
...,...,...,...,...,...
31923236,200443,287639,2.5,1695648354,1098110.0
31923238,200443,289993,3.5,1696530008,1120628.0
31960126,200678,287699,4.5,1692118588,872585.0
31976753,200774,287633,2.5,1693098259,747188.0


In [18]:
print(f'Min time: {df['timestamp'].min()}')
print(f'Max time: {df['timestamp'].max()}')

print(f'Unique movies: {df["movieId"].nunique()}')
print(f'Unique users: {df["userId"].nunique()}')
print(f'Number of ratings: {len(df)}')

Min time: 1691884909
Max time: 1697163675
Unique movies: 659
Unique users: 2305
Number of ratings: 7663


In [19]:
df['tmdbId'] = df['tmdbId'].fillna(0).astype(int)
df['userId'] = 'U' + df['userId'].astype(str)

C:\Users\minhc\AppData\Local\Temp\ipykernel_11492\2758115749.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tmdbId'] = df['tmdbId'].fillna(0).astype(int)
C:\Users\minhc\AppData\Local\Temp\ipykernel_11492\2758115749.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['userId'] = 'U' + df['userId'].astype(str)


Get data from TMDB API to build website recommendation

In [ ]:
from dotenv import dotenv_values

config = dotenv_values(".env")
TMDB_API_KEY = config.get("TMDB_API_KEY")

In [ ]:
def get_movie_details(movie_id):
    API_KEY = TMDB_API_KEY
    BASE_URL = 'https://api.themoviedb.org/3'
    DETAILS_ENDPOINT = f'{BASE_URL}/movie/{{movie_id}}'
    response = requests.get(DETAILS_ENDPOINT.format(movie_id=movie_id), params={'api_key': API_KEY})

    if response.status_code == 200:
        data = response.json()
        return {
            "tmdbId": data.get("id"),
            "title": data.get("title"),
            "poster": f"https://image.tmdb.org/t/p/w500{data.get('poster_path')}",
            "date_published": data.get("release_date"),
            "homepage": data.get("homepage")
        }
    else:
        None

####################

tmdb_movie = []
tmdbid_list = df['tmdbId'].unique()
counter = 0

for tmdbid in tmdbid_list:
    movie = get_movie_details(tmdbid)
    if movie:
        tmdb_movie.append(movie)
        counter += 1  
    
    if counter % 50 == 0:
        print(f"{counter}/{len(tmdbid_list)} movies processed...")
    # Add a delay to handle API rate limits
    time.sleep(0.25) 

clear_output(wait=False)
print(f"Processed {counter}/{len(tmdbid_list)} movies.")

tmdb_movie = pd.DataFrame(tmdb_movie)


Processed 655/659 movies.


In [21]:
df = pd.merge(df, tmdb_movie, on='tmdbId')
df.drop(columns=['movieId'], inplace=True)

df.to_csv('Dataset.csv', index=False)

df

,userId,rating,timestamp,tmdbId,title,poster,date_published,homepage
0,U28,3.0,1695439205,447365,Guardians of the Galaxy Vol. 3,https://image.tmdb.org/t/p/w500/r2J02Z2OpNTctf...,2023-05-03,https://www.marvel.com/movies/guardians-of-the...
1,U28,5.0,1692916119,569094,Spider-Man: Across the Spider-Verse,https://image.tmdb.org/t/p/w500/8Vt6mWEReuy4Of...,2023-05-31,https://www.acrossthespiderverse.movie
2,U28,5.0,1692916114,872585,Oppenheimer,https://image.tmdb.org/t/p/w500/8Gxv8gSFCU0XGD...,2023-07-19,https://www.universalpictures.com/movies/oppen...
3,U28,5.0,1693533442,346698,Barbie,https://image.tmdb.org/t/p/w500/iuFNMS8U5cb6xf...,2023-07-19,https://www.barbie-themovie.com
4,U28,4.0,1696391189,1059811,Flora and Son,https://image.tmdb.org/t/p/w500/pq2p8ovf8PZps2...,2023-09-21,https://tv.apple.com/movie/umc.cmc.1ol2idv8q2i...
...,...,...,...,...,...,...,...,...
7651,U200443,2.5,1695648354,1098110,Blood & Gold,https://image.tmdb.org/t/p/w500/oLRQP5cEjiT1Dx...,2023-04-21,https://www.netflix.com/title/81557402
7652,U200443,3.5,1696530008,1120628,Hard Feelings,https://image.tmdb.org/t/p/w500/d2jU7XWkPjRN0L...,2023-05-24,https://www.netflix.com/title/81580024
7653,U200678,4.5,1692118588,872585,Oppenheimer,https://image.tmdb.org/t/p/w500/8Gxv8gSFCU0XGD...,2023-07-19,https://www.universalpictures.com/movies/oppen...
7654,U200774,2.5,1693098259,747188,Asteroid City,https://image.tmdb.org/t/p/w500/hfo7pvL9Fys7ro...,2023-06-08,https://www.focusfeatures.com/asteroid-city


In [22]:
del movies
del ratings
del links
del df
del tmdb_movie